In [3]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Streaming de Ficheros") \
        .master("local[2]") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 3) \
        .getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
esquema = StructType([
    StructField("Nombre", StringType()),
    StructField("Cantidad", IntegerType()),
    StructField("Concepto", StringType())
])

In [6]:
bizumsDF = spark.readStream.option("sep",";").schema(esquema).csv("bizums/recibidos")

In [7]:
mayorDF = bizumsDF.groupBy("Nombre", "Concepto").agg({"Cantidad":"max"})

In [8]:
mayorDF.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Concepto: string (nullable = true)
 |-- max(Cantidad): integer (nullable = true)



In [9]:
# Mostramos las palabras por consola (sink)
bizumQuery = mayorDF.writeStream \
        .format("console") \
        .outputMode("complete") \
        .start()

In [ ]:
bizumQuery.awaitTermination()